# Búsqueda avanzada

En esta práctica vamos a trabajar con las búsquedas textuales, para ello vamos a tener que trabajar tanto con el comando search como con los mapping types. El objetivo es poder crear cualquier buscador documental.

## Introducción

Este primer bloque de código sirve para configurar el Notebook

In [ ]:
from IPython.display import JSON

Ahora vamos a descargar el cliente de ElasticSearch en Python.

In [ ]:
pip install elasticsearch==7.10.1

Por último, creamos la conexion con el servidor de Elastic Search desplegado

In [ ]:
from elasticsearch import Elasticsearch
es = Elasticsearch(
    ['elasticsearch']
)
JSON(es.info())

## Importando los datos

En primer lugar vamos a descargar los datos usando el comando:

In [ ]:
!wget "https://gist.githubusercontent.com/aagea/76a7e86ee77c95b09413d32d48f6af90/raw/0d107ae09c388f498bc7eefeebb79a27de0ec46c/imdb.json"

In [ ]:
es.indices.delete(index="imdb",ignore=[400,404])
!curl -H "Content-Type: application/json" -XPOST "http://elasticsearch:9200/imdb/_bulk?pretty" --data-binary "@imdb.json" >> /dev/null;

Ahora vamos a recuperar a los 10 primeros hoteles que hemos indexado.

In [ ]:
JSON(es.search(index="imdb"))

Por otro lado vamos a revisar el mapping type que se ha generado.

In [ ]:
JSON(es.indices.get_mapping(index="imdb"))

## Ejercicio 1

Revisa el mapping type auto generado y modificalo para que se adecue mejor a los datos almacenados. Guardalo en la variable mapping_type. Ver [Mapping data types](https://www.elastic.co/guide/en/elasticsearch/reference/7.17/mapping-types.html)

In [ ]:
#Este código es de ejemplo
mapping_type = {
    'mappings': {
        'properties': {
            'actors': {
                'type': 'text',
                'fields': {
                    'keyword': {
                        'type': 'keyword', 
                        'ignore_above': 256
                    }
                }
            },
            'averageRating': {
                'type': 'long'
            },
            'contentRating': {
                'type': 'text',
                'fields': {
                    'keyword': {
                        'type': 'keyword',
                        'ignore_above': 256
                    }
                }
            },
            'duration': {
                'type': 'text',
                'fields': {
                    'keyword': {
                        'type': 'keyword', 
                        'ignore_above': 256
                    }
                }
            },
            'genres': {
                'type': 'text',
                'fields': {
                    'keyword': {
                        'type': 'keyword', 
                        'ignore_above': 256
                    }
                }
            },
            'imdbRating': {
                'type': 'float'
            },
            'originalTitle': {
                'type': 'text',
                'fields': {
                    'keyword': {
                        'type': 'keyword', 
                        'ignore_above': 256
                    }
                }
            },
            'poster': {
                'type': 'text',
                'fields': {
                    'keyword': {
                        'type': 'keyword',
                        'ignore_above': 256
                    }
                }
            },
            'posterurl': {
                'type': 'text',
                'fields': {
                    'keyword': {
                        'type': 'keyword', 
                        'ignore_above': 256
                    }
                }
            },
            'ratings': {
                'type': 'long'
            },
            'releaseDate': {
                'type': 'date'
            },
            'storyline': {
                'type': 'text',
                'fields': {
                    'keyword': {
                        'type': 'keyword', 
                        'ignore_above': 256
                    }
                }
            },
            'title': {
                'type': 'text',
                'fields': {
                    'keyword': {
                        'type': 'keyword', 
                        'ignore_above': 256
                    }
                }
            },
            'year': {
                'type': 'text',
                'fields': {
                    'keyword': {
                        'type': 'keyword', 
                        'ignore_above': 256
                    }
                }
            }
        }
    }
}
es.indices.delete(index="imdb",ignore=[400,404])
es.indices.create(index="imdb",body=mapping_type)

Cuando hayas terminado ejecuta este comando.

In [ ]:
!curl -H "Content-Type: application/json" -XPOST "http://elasticsearch:9200/imdb/_bulk?pretty" --data-binary "@imdb.json" 

Revisa que el mapping type es correcto.

In [ ]:
JSON(es.indices.get_mapping(index="imdb"))

## Analyzers

Los [analyzers](https://www.elastic.co/guide/en/elasticsearch/reference/7.17/configure-text-analysis.html) permiten normalizar y estandarizar los datos introducidos por el usuario. Para probar los resultados de un analizador podemos utilizar la función `analyzer`.

In [ ]:
body_request={
  "analyzer": "standard",
  "text": "The 2 QUICK Brown-Foxes jumped over the lazy dog's bone."
}
JSON(es.indices.analyze(body=body_request))

Los [tokenizers](https://www.elastic.co/guide/en/elasticsearch/reference/7.17/analysis-tokenizers.html) son una entidad de más bajo nivel que recibe un stream de caracteres y devuelve un listado de tokens. En este caso el tokenicer keyword, lo que hace es crear un único toke con el testo completo. 

[Filter](https://www.elastic.co/guide/en/elasticsearch/reference/7.17/analysis-tokenfilters.html) permite modificar los tokens generados por el tokenizer. 

[Char filter](https://www.elastic.co/guide/en/elasticsearch/reference/7.17/analysis-charfilters.html) permite remplazar texto antes de de ser procesado por el tokenizer.

In [ ]:

body_request={
  "tokenizer" : "keyword",
  "filter" : ["lowercase"],
  "char_filter" : ["html_strip"],
  "text" : "this is a <b>test</b>"
}
JSON(es.indices.analyze(body=body_request))

## Creando sinonimos

Vamos a ver como utilizando el `char_filter`podemos modificar el analizador para crear sinonimos. Vamos a sustituir los emoticonos `:)` por `_happy_face_` y `:(` por `_sad_face_`.


Para ello vamos a utilizar el `char_filter`de tipo [mapping](https://www.elastic.co/guide/en/elasticsearch/reference/7.17/analysis-mapping-charfilter.html).

In [ ]:
body_request={
    "tokenizer" : "keyword",
    "char_filter": { 
          "type": "mapping",
          "mappings": [
            ":) => _happy_face_",
            ":( => _sad_face_"
          ]        
    },
    "text" : "Are you :(? No, I am :)"
}
JSON(es.indices.analyze(body=body_request))

Ahora podemos crear un indice que contega este char filter.

In [ ]:
es.indices.delete(index="emojis",ignore=[400,404])
body_request= {
    "settings": {
        "analysis": {
            "analyzer":{
                "emojis":{
                    "tokenizer": "standard",
                    "char_filter":["emojis_filter"]
                }
            },
            "char_filter":
                {
                    "emojis_filter": {
                        "type": "mapping",
                        "mappings": [
                            ":) => _happy_face_",
                            ":( => _sad_face_"
                        ]
                    }
                }
            
        }
    },
    "mappings": {
        "properties": {
            "msg": {
                "type": "text",
                "analyzer":"emojis",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                        "ignore_above": 256
                    }
                }
            }
        }
    }
}

es.indices.create(index="emojis",body=body_request)

Comprobemos que el analizador funciona.

In [ ]:
request_body={
  "analyzer": "emojis",
  "text" : "He who controls the spice, controls the universe :), Dune"
}
JSON(es.indices.analyze(index="emojis",body=request_body))

Podemos indexar un documento y chequear que se puede buscar utilzando el sinonimo.

In [ ]:
JSON(es.index(index="emojis", body={"msg":"Are you :(? No, I am :)"}))
request_body={
  "query": {
    "match": {
      "msg": "_happy_face_"
    }
  }
}
JSON(es.search(index="emojis", body=request_body))

## Ejercicio 2

Modifica el mapping type de imdb y re-indexa el contenido para incluir los siguiente sinomos en el campo `actors`.

* Brad Pitt -> William Bradley Pitt
* Meat Loaf -> Michael Lee Aday
* Tom Hanks -> Thomas Jeffrey Hanks
* John Travolta -> John Joseph Travolta
* Uma Thurman -> Uma Karuna Thurman

## Otros tipos de consulta
ElasticSearch nos da otros tipos de consulta que permiten mejora la experiencia del usuario.

### More like this

El [more like this](https://www.elastic.co/guide/en/elasticsearch/reference/7.17/query-dsl-mlt-query.html) encuentra documentos que son parecido a otros documentos dados.

In [ ]:
request_body={
  "query": {  
      "more_like_this" : {
          "fields" : ["originalTitle"],
          "like" : ["The Godfather"],
          "min_term_freq" : 1,
          "min_doc_freq": 1,  
          "minimum_should_match": "10%"
        }
  }
}
JSON(es.search(index="imdb", body=request_body))

## Suggesters
Los [suggesters](https://www.elastic.co/guide/en/elasticsearch/reference/7.17/search-suggesters.html) nos permiten dar recomendaciones de busqueda utilizando un texto. Es útil para resolver problemas antes de que se lance la búsqueda.

In [ ]:
request_body={
    "suggest": {
        "my-suggestion" : {
            "text" : "godfater",
            "term" : {
                "field" : "originalTitle"
            }
        }
    }
}
JSON(es.search(index="imdb", body=request_body))

## Ejercicio 3

Calcula la media de ratings por cada actor

Cuantas peliculas se hicerion antes 1990, 1990-2000, 2000+

Encuentra todas las peliculas que empiece por "Lord of the Ring"